In [5]:
"""
Add datepicker

Change slider from range to ordinary using int timestamps as steps, possibly without tooltips
(or dynamically transform value to datetime)

https://bokeh.pydata.org/en/latest/docs/user_guide/extensions_gallery/widget.html

Transform p2 to something more informative

"""

import bokeh
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
from bokeh.models import ColumnDataSource, DateRangeSlider, DatePicker, BoxAnnotation, Slider, Select
from bokeh.layouts import layout, row, column, Spacer

from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler

output_notebook()

min_border_left = 50
plot_height = 600
plot_width = 1000
scale = 0.5
offset = 79

class Specgram():   
        
    def __init__(self, start_dt):
        
        self.init_date = start_dt
        self.data = self._load_data(start_dt)
        self.rng = self._get_rng(start_dt, start_dt + timedelta(seconds=1))
        
        self.beams = np.arange(1, self.data.shape[1] + 1).astype(str)
        
    def _load_data(self, date):
        """Load data file for the specified date, set visible range"""
        nsamples = 30*80
        nbeams = 48
        mean = 0.5
        data = np.random.rand(nsamples, nbeams) - mean
        return data
    
    def _get_rng(self, start_dt, end_dt):
        return pd.date_range(start=start_dt, end=end_dt, periods=self.data.shape[0])

    def _gui(self, doc):
        # Source
        source = ColumnDataSource({'xs':[self.rng]*len(self.beams), 
                                   'ys':list(zip(*[zip(self.beams, x) for x in self.data*scale]))})

        # Main figure
        tools = 'hover, box_zoom, wheel_zoom, reset, save'
        p1 = figure(min_border_left=min_border_left,
                    plot_height=plot_height,
                    plot_width=plot_width, 
                    x_axis_type='datetime',
                    x_range=(self.rng[0], self.rng[offset]),
                    y_range=self.beams,
                    tools=tools)
        p1.toolbar.logo = None

        # Slider figure
        p2 = figure(plot_height=plot_height//12,
                    plot_width=plot_width,
                    x_axis_type='datetime',
                    x_range=(self.rng[0], self.rng[-1]),
                    y_range=(1, 2),
                    toolbar_location=None)
        p2.yaxis.major_label_text_color = None
        p2.yaxis.major_tick_line_color= None
        p2.yaxis.minor_tick_line_color= None
        p2.grid.grid_line_color=None

        # Datepicker widget
        def update_date(attr, old, new):
            self.data = self._load_data(new)
            self.rng += (new - self.rng[0].date())
            source.data={'xs':[self.rng]*len(self.beams), 
                         'ys':list(zip(*[zip(self.beams, i) for i in self.data*scale]))}
            p2.x_range.start = rslider.start = self.rng[0]
            p2.x_range.end = rslider.end = self.rng[-1]
            rslider.value = (self.rng[0], self.rng[offset])
            
        datepicker = DatePicker(title='Дата',
                                value=self.rng[0])
        datepicker.on_change('value', update_date)
        
        # Hourselect widget
        def update_time(attr, old, new):
            print(attr)
            
        hourselect = Select(options=['-'] + list(map('{:02d}'.format, range(0, 24))),
                            value='-')
        hourselect.on_change('value', update_time)

        # Slider widget
        def update_range(attr, old, new):
            box.left = new[0]
            box.right = new[1]
            p1.x_range.start = new[0]
            p1.x_range.end = new[1]
            
        rslider = DateRangeSlider(start=self.rng[0], 
                                  end=self.rng[-1], 
                                  value=(self.rng[0], self.rng[offset]),
                                  title=None,
                                  show_value=True,
                                  tooltips=True,
                                  width=plot_width-77)
        rslider.on_change('value', update_range)
        spacer_edit = Spacer(width=40)

        # Box selection
        box = BoxAnnotation(fill_alpha=0.5,
                            line_alpha=0.5,
                            level='underlay',
                            left=self.rng[0],
                            right=self.rng[offset])

        # Plot
        p1.multi_line('xs', 'ys', source=source, alpha=0.6)

        # Layout
        p2.add_layout(box)
        doc.add_root(layout(row(datepicker, hourselect), p1, p2, row(spacer_edit, rslider)))
        
        return doc
    
    def show(self):
        handler = FunctionHandler(self._gui)
        app = Application(handler)
        show(app)
        
specgram = Specgram(datetime.strptime('2016-10-27', '%Y-%m-%d'))
specgram.show()

Loading BokehJS ...

In [4]:
import numpy as np

from bokeh.plotting import figure, show

N = 10000

x = np.linspace(0, 10*np.pi, N)
y = np.cos(x) + np.sin(2*x+1.25) + np.random.normal(0, 0.001, (N, ))

p = figure(title="A line consisting of 10k points", output_backend="webgl")
p.line(x, y, color="#22aa22", line_width=3)
show(p)